In [1]:
import os
import json
import gc
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
from transformers import AutoTokenizer
from typing import Dict, List, Set, Tuple, NamedTuple, Callable
import scipy
import scml
from scml import pandasx as pdx
from daigt.preprocess import en as pen
tim = scml.Timer()
tim.start()
os.environ["TOKENIZERS_PARALLELISM"] = "false"
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
scml.seed_everything()
info = np.iinfo(np.int16)
print(f"int16, min={info.min}, max={info.max}")

int16, min=-32768, max=32767


In [2]:
min_len, max_len = 773, 32_767  # P01 cutoff

In [3]:
#tokenizer = AutoTokenizer.from_pretrained("huggingface/microsoft/deberta-v3-base", is_fast=True)
#print(f"{repr(tokenizer)}\nmodel_input_names={tokenizer.model_input_names}")

# Combine data sources

In [4]:
edf = pd.read_csv("input/train_essays.csv", low_memory=False)
edf = edf.rename(columns={"id": "essay_id"})
edf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1378 entries, 0 to 1377
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   essay_id   1378 non-null   object
 1   prompt_id  1378 non-null   int64 
 2   text       1378 non-null   object
 3   generated  1378 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 43.2+ KB


In [5]:
pdf = pd.read_csv("input/train_prompts.csv", low_memory=False)
pdf = pdf.set_index("prompt_id")
pdf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2 entries, 0 to 1
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   prompt_name   2 non-null      object
 1   instructions  2 non-null      object
 2   source_text   2 non-null      object
dtypes: object(3)
memory usage: 64.0+ bytes


In [6]:
df1 = pd.read_csv("input/train_drcat_04.csv", low_memory=False)
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44206 entries, 0 to 44205
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   essay_id  44206 non-null  object
 1   text      44206 non-null  object
 2   label     44206 non-null  int64 
 3   source    44206 non-null  object
 4   prompt    12911 non-null  object
 5   fold      44206 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 2.0+ MB


In [7]:
df2 = pd.read_csv("input/train_external_drcat_02.csv", low_memory=False)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195603 entries, 0 to 195602
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   essay_id  195603 non-null  object 
 1   text      195603 non-null  object 
 2   label     7074 non-null    float64
 3   source    195603 non-null  object 
 4   prompt    6671 non-null    object 
dtypes: float64(1), object(4)
memory usage: 7.5+ MB


In [8]:
def prepare_dataset(df)->pd.DataFrame:
    cols = ["essay_id", "text", "label", "source", "prompt"]
    df = df[cols]
    df = df.rename(columns={"label": "generated"})
    df["prompt"] = df["prompt"].fillna("na")
    df["generated"] = df["generated"].fillna(0)
    return df


df1 = prepare_dataset(df1)
df2 = prepare_dataset(df2)

In [9]:
def get_prompt(row)->str:
    p=pdf.loc[row["prompt_id"]]
    name=p["prompt_name"]
    inst=p["instructions"]
    return f"{name} {inst}"
    

edf["prompt"] = edf.progress_apply(get_prompt, axis=1)
edf["source"] = "train_essays"
df = pd.concat([edf, df1, df2], ignore_index=True)
cols = ["essay_id", "generated", "source", "prompt", "text"]
df = df[cols].copy()
cols = ["generated"]
df[cols] = df[cols].astype(np.int8)
df.info()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1378/1378 [00:00<00:00, 76288.26it/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241187 entries, 0 to 241186
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   essay_id   241187 non-null  object
 1   generated  241187 non-null  int8  
 2   source     241187 non-null  object
 3   prompt     241187 non-null  object
 4   text       241187 non-null  object
dtypes: int8(1), object(4)
memory usage: 7.6+ MB


In [10]:
more = len(df)
df = df.drop_duplicates(["text"], ignore_index=True)
print(f"{more - len(df)} rows dropped: duplicates")

4000 rows dropped: duplicates


# Preprocess Text

In [11]:
bsc = pen.BasicPreprocessor()
bow = pen.BowPreprocessor()

def preprocess_bsc(col) -> Callable:
    def fn(row) -> str:
        return bsc(row[col])
    
    return fn

def preprocess_bow(col) -> Callable:
    def fn(row) -> str:
        return bow(row[col], drop_stopword=False)
    
    return fn

In [12]:
cols = ["text", "prompt"]
for col in cols:
    print(col)
    new_col = f"{col}_bsc" 
    df[new_col] = df.progress_apply(preprocess_bsc(col), axis=1)
    new_col = f"{col}_bow" 
    df[new_col] = df.progress_apply(preprocess_bow(col), axis=1)
    col1, col2 = new_col, f"{col}_bow_len"
    df[col2] = df[col1].str.len()
    df[col2] = df[col2].astype(np.int16)
    #x = tokenizer(df[new_col].tolist(), truncation=False, add_special_tokens=False)
    #len_col = f"{new_col}_len" 
    #df[len_col] = [len(s) for s in x["input_ids"]]
    #df[len_col] = df[len_col].astype(np.int16) 

text


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 237187/237187 [50:24<00:00, 78.42it/s]


prompt


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 237187/237187 [00:23<00:00, 10263.45it/s]


# Filter Invalid Data

In [13]:
more = len(df)
df = df.drop_duplicates(["text_bow"], ignore_index=True)
print(f"{more - len(df)} rows dropped: duplicates")

278 rows dropped: duplicates


In [14]:
df.describe(percentiles=percentiles)

,generated,text_bow_len,prompt_bow_len
count,236909.000000,236909.000000,236909.000000
mean,0.079545,5245.604895,29.109671
std,0.270589,5819.164365,97.497809
min,0.000000,-32766.000000,2.000000
1%,0.000000,60.000000,2.000000
5%,0.000000,574.000000,2.000000
10%,0.000000,1022.000000,2.000000
20%,0.000000,1608.000000,2.000000
30%,0.000000,2082.000000,2.000000
40%,0.000000,2701.000000,2.000000


In [15]:
more = len(df)
df = df[(df["text_bow_len"]>=min_len) & (df["text_bow_len"]<=max_len)].copy()
print(f"{more - len(df):,} rows dropped: text length must be between {min_len} and {max_len}")

15297 rows dropped: text length must be between 773 and 32767


# Review Data

In [18]:
df = df.reset_index(drop=True) 
col = "row_id"
df[col] = df.index
df[col] = df[col].astype(np.int32)
cols = list(df.columns)
cols = [cols[-1]] + cols[:-1]
df = df[cols]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221612 entries, 0 to 221611
Data columns (total 13 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   row_id          221612 non-null  int32 
 1   index           221612 non-null  int64 
 2   essay_id        221612 non-null  object
 3   generated       221612 non-null  int8  
 4   source          221612 non-null  object
 5   prompt          221612 non-null  object
 6   text            221612 non-null  object
 7   text_bsc        221612 non-null  object
 8   text_bow        221612 non-null  object
 9   text_bow_len    221612 non-null  int16 
 10  prompt_bsc      221612 non-null  object
 11  prompt_bow      221612 non-null  object
 12  prompt_bow_len  221612 non-null  int16 
dtypes: int16(2), int32(1), int64(1), int8(1), object(8)
memory usage: 17.1+ MB


In [20]:
df.describe(percentiles=percentiles)

,row_id,generated,text_bow_len,prompt_bow_len
count,221612.000000,221612.000000,221612.000000,221612.000000
mean,110805.500000,0.081706,5747.756854,29.997636
std,63974.018265,0.273917,5310.914738,99.333283
min,0.000000,0.000000,773.000000,2.000000
1%,2216.110000,0.000000,855.000000,2.000000
5%,11080.550000,0.000000,1090.000000,2.000000
10%,22161.100000,0.000000,1368.000000,2.000000
20%,44322.200000,0.000000,1867.000000,2.000000
30%,66483.300000,0.000000,2323.000000,2.000000
40%,88644.400000,0.000000,3107.000000,2.000000


In [21]:
cols=["generated"]
pdx.value_counts(df[cols])

,count,percent
generated,,
0,203505,0.918294
1,18107,0.081706


In [22]:
cols=["prompt_bow"]
pdx.value_counts(df[cols]).head(50)

,count,percent
prompt_bow,,
na,201648,0.909915
car free cities write an explanatory essay to inform fellow citizens about the advantages of limiting car usage your essay must be based on ideas and information that can be found in the passage set manage your time carefully so that you can read the passages plan your response write your response and revise and edit your response be sure to use evidence from multiple sources and avoid overly relying on one source your response should be in the form of a multiparagraph essay write your essay in the space provided,708,0.003195
does the electoral college work write a letter to your state senator in which you argue in favor of keeping the electoral college or changing to election by popular vote for the president of the united states use the information from the texts in your essay manage your time carefully so that you can read the passages plan your response write your response and revise and edit your response be sure to include a claim address counterclaims use evidence from multiple sources and avoid overly relying on one source your response should be in the form of a multiparagraph essay write your response in the space provided,670,0.003023
it is better to have broad knowledge of many academic subjects than to specialize in one specific subject do you agree or disagree use specific reasons and examples to support your answer write an essay of roughly 400 words,189,0.000853
most advertisements make products seem much better than they really are do you agree or disagree use specific reasons and examples to support your answer write an essay of roughly 400 words,188,0.000848
young people nowadays do not give enough time to helping their communities do you agree or disagree use specific reasons and examples to support your answer write an essay of roughly 400 words,187,0.000844
successful people try new things and take risks rather than only doing what they already know how to do well do you agree or disagree use specific reasons and examples to support your answer write an essay of roughly 400 words,186,0.000839
it is more important for students to understand ideas and concepts than it is for them to learn facts do you agree or disagree use specific reasons and examples to support your answer write an essay of roughly 400 words,179,0.000808
in twenty years there will be fewer cars in use than there are today do you agree or disagree use specific reasons and examples to support your answer write an essay of roughly 400 words,178,0.000803


In [23]:
cols=["source"]
pdx.value_counts(df[cols])

,count,percent
source,,
qwedsacf/ivypanda-essays,124725,0.562808
persuade_corpus,25793,0.116388
nid989/EssayFroum-Dataset,23914,0.107909
dim/essayforum_raw_writing_10k,18314,0.082640
sakibsh-llm-human-1,2459,0.011096
mistral7binstruct_v1,2420,0.010920
llama2_chat,2418,0.010911
mistral7binstruct_v2,2409,0.010870
original_moth,2264,0.010216


In [24]:
df.groupby(["source", "generated"])["essay_id"].count()

source                                       generated
ChristophSchuhmann/essays-with-instructions  0              2025
argu-gpt-3.5-turbo                           1               812
argu-gpt2-xl                                 1               170
argu-text-babbage-001                        1               362
argu-text-curie-001                          1               471
argu-text-davinci-001                        1               502
argu-text-davinci-002                        1               498
argu-text-davinci-003                        1               806
chat_gpt_moth                                1              2124
darragh_claude_v6                            1              1000
darragh_claude_v7                            1              1000
dim/essayforum_raw_writing_10k               0             18314
falcon_180b_v1                               1              1051
llama2_chat                                  1              2418
llama_70b_v1                       

In [25]:
df.sample(5).head()

row_id  \
61418    61418   
126529  126529   
79554    79554   
144421  144421   
23428    23428   

                                                                          essay_id  \
61418       https://ivypanda.com/essays/impact-of-covid-19-on-the-retail-industry/   
126529  https://ivypanda.com/essays/gender-and-educational-level-chi-square-study/   
79554                      https://ivypanda.com/essays/caring-for-a-family-member/   
144421          https://ivypanda.com/essays/surrogacy-and-how-it-affects-families/   
23428                                                                 C044BD1DAC0E   

        generated                    source prompt  \
61418           0  qwedsacf/ivypanda-essays     na   
126529          0  qwedsacf/ivypanda-essays     na   
79554           0  qwedsacf/ivypanda-essays     na   
144421          0  qwedsacf/ivypanda-essays     na   
23428           0           persuade_corpus     na   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [26]:
%%time
df.to_parquet(f"output/preprocess.parquet", index=False)
assert df.notna().all(axis=None)

CPU times: user 8.04 s, sys: 821 ms, total: 8.86 s
Wall time: 9.04 s


In [27]:
tim.stop()
print(f"Total time taken {str(tim.elapsed)}")

Total time taken 1:01:14.594538
